# Análisis de correlación

Un medida útil para evaluar el grado de relación lineal entre dos variables cuantitativas es el coeficiente de correlación de Pearson. Este coeficiente puede tomar valores entre -1 y 1, donde valores positivos indican una relación directa, valores negativos indican una relación inversa, y cuanto más cercano sea a 0 más débil es esta relación. 

Para calcular un coeficiente de correlación de Pearson entre dos columnas se puede utilizar el método *corr()* de Pandas. Por ejemplo:
`correlacion = df['X'].corr(df['Y'])`  

Para calcular el coeficiente de correlación de Spearman se puede agregar como argumento:  
`correlacion = df['X'].corr(df['Y'], method='spearman')`  

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_excel('https://github.com/adan-rs/AnalisisDatos/raw/main/datasets/enigh2020.xlsx')

Eliminemos datos atípicos

In [ ]:
from sklearn.ensemble import IsolationForest

def remove_outliers_iso_forest(df, columns, contamination=0.05, random_state=42):
    """
    Elimina valores atípicos utilizando el algoritmo Isolation Forest.
    Retorna DataFrame sin valores atípicos.
    """
    # Initialize and fit Isolation Forest model
    iso_forest = IsolationForest(contamination=contamination, random_state=random_state)
    iso_forest.fit(df[columns])
    
    # Predict labels: 1 (normal) or -1 (outlier)
    labels = iso_forest.predict(df[columns])
    
    # Calculate and display number of excluded outliers
    df_clean = df[labels == 1]
    excluded_values = len(df) - len(df_clean)
    print(f"\nCantidad de valores atípicos excluidos: {excluded_values}")
    return df_clean

In [ ]:
df = remove_outliers_iso_forest(df, ['ing_cor', 'gasto_mon'])

In [ ]:
df.plot(kind='scatter', x='ing_cor', y='gasto_mon');

El coeficiente de correlación es una medida del grado o fuerza de la relación lineal entre dos variables cuantitativas. Un valor positivo del coeficiente de correlación indica una relación directa mientras que un valor negativo indica una relación inversa.

In [ ]:
# Calcula la correlación entre "ing_cor" y "gasto_mon"
correlacion = df['ing_cor'].corr(df['gasto_mon'])
correlacion.round(4)

Si se tiene un DataFrame *df* de variables cuantitativas se puede obtener la matriz de correlaciones utilizando `df.corr()`

In [ ]:
var_cont = ["ing_cor", "gasto_mon", "edad_jefe", "tot_integ"]
matriz_corr = df[var_cont].corr()
matriz_corr

In [ ]:
fig, ax = plt.subplots(figsize=(6, 4))
ax = sns.heatmap(matriz_corr, annot=True, cmap='Greens')
ax.set_title('Mapa de calor de las correlaciones');

## Prueba de hipótesis para la correlación

*¿Para qué se utiliza?* Se utiliza para evaluar si existe una relación o no entre dos variables cuantitativas. Esta prueba evalúa si el coeficiente de correlación de Pearson (r) entre las dos variables es significativamente diferente de cero.

*Variables*. Se consideran dos variables cuantitativas con escala de medición de razón o de intervalo.

*Hipótesis*. Las hipótesis son:
- Hipótesis nula (H0):  No existe una correlación entre las dos variables (r = 0).
- Hipótesis alternativa (H1): Existe una correlación entre las dos variables (r ≠ 0).

*Supuestos principales* 
- La relación entre ambas variables debe ser lineal.
- La distribución de los datos debe ser normal (o una muestra grande)

*Procedimiento*. Se recomienda utilizar la biblioteca *scipy.stats* debido a que permite obtener el valor p. Para ello se puede usar:

```
corr_coef, p_valor = pearsonr(df['X'], df['Y'])  
print('Coeficiente de correlación: ', corr_coef)  
print('Valor p', p_valor)
```

In [ ]:
from scipy.stats import pearsonr

def prueba_correlacion_pearson(df, var1, var2, alfa=0.05, imprimir=True):
    """
    Realiza una prueba de correlación de Pearson entre dos variables numéricas.
    Retorna un diccionario con los resultados de la prueba.
    """
    # Verificar condiciones de aplicabilidad
    if var1 not in df.columns or var2 not in df.columns:
        raise ValueError(f"Una o ambas variables no existen en el DataFrame: {var1}, {var2}")    
    if not np.issubdtype(df[var1].dtype, np.number) or not np.issubdtype(df[var2].dtype, np.number):
        raise ValueError("Ambas variables deben ser numéricas para calcular la correlación de Pearson")
    datos_completos = df[[var1, var2]].dropna()
    
    # Realizar prueba de correlación
    correlacion, p_valor = pearsonr(datos_completos[var1], datos_completos[var2])
        
    # Determinar resultado
    resultado = ('Existe una correlación significativa entre ambas variables' 
                 if p_valor < alfa 
                 else 'No existe una correlación significativa entre ambas variables')

    # Interpretar la fuerza de la correlación
    interpretacion = ("Correlación muy fuerte" if abs(correlacion) >= 0.9 else
                      "Correlación fuerte" if abs(correlacion) >= 0.7 else
                      "Correlación moderada" if abs(correlacion) >= 0.5 else
                      "Correlación débil" if abs(correlacion) >= 0.3 else
                      "Correlación nula o muy débil")
    interpretacion += " (negativa)" if correlacion < 0 else " (positiva)" if correlacion > 0 else ""

    # Imprimir resultados
    print(f'\nCorrelación entre {var1} y {var2}:')
    print(f'Coeficiente de correlación: {correlacion:.4f}')
    print(f'Valor p: {p_valor:.4f}')
    print(f'Interpretación: {interpretacion}')
    print(f'Resultado: {resultado}')

    # Retornar resultados en un diccionario
    return {'correlacion': correlacion,
            'p_valor': p_valor,
            'interpretacion': interpretacion,
            'resultado': resultado}

In [ ]:
resultados = prueba_correlacion_pearson(df, 'ing_cor', 'gasto_mon')

Ejemplo de reporte de metodología y resultados:
> “Se realizó una prueba de correlación de Pearson para evaluar la relación entre el ingreso corriente y el gasto monetario. Se encontró una correlación positiva significativa entre ambas variables (r = 0.6354. p < 0.001)”

## Comentarios adicionales
El coeficiente de correlación de Spearman es una alternativa apropiada cuando las variables son ordinales, existen relaciones monótonas no lineales, o la distribución no es normal. 
```
from scipy.stats import spearmanr
corr_coef, p_value = spearmanr(x, y)
```

In [ ]:
from scipy.stats import spearmanr

def prueba_correlacion_spearman(df, var1, var2, alfa=0.05):
    """
    Realiza una prueba de correlación de Spearman entre dos variables.
    Retorna un diccionario con los resultados de la prueba.
    """
    # Verificar condiciones de aplicabilidad
    if var1 not in df.columns or var2 not in df.columns:
        raise ValueError(f"Una o ambas variables no existen en el DataFrame: {var1}, {var2}")
    datos_completos = df[[var1, var2]].dropna()
    
    # Realizar prueba de correlación
    correlacion, p_valor = spearmanr(datos_completos[var1], datos_completos[var2])
    
    # Determinar resultado e interpretación
    resultado = ('Existe una correlación significativa entre ambas variables' 
                if p_valor < alfa 
                else 'No existe una correlación significativa entre ambas variables')
    
    interpretacion = ("Correlación muy fuerte" if abs(correlacion) >= 0.9 else
                     "Correlación fuerte" if abs(correlacion) >= 0.7 else
                     "Correlación moderada" if abs(correlacion) >= 0.5 else
                     "Correlación débil" if abs(correlacion) >= 0.3 else
                     "Correlación nula o muy débil")
    interpretacion += " (negativa)" if correlacion < 0 else " (positiva)" if correlacion > 0 else ""
    
    # Imprimir resultados
    print(f'\nCorrelación de Spearman entre {var1} y {var2}:')
    print(f'Coeficiente de correlación: {correlacion:.4f}')
    print(f'Valor p: {p_valor:.4f}')
    print(f'Interpretación: {interpretacion}')
    print(f'Resultado: {resultado}')
    
    return {
        'correlacion': correlacion,
        'p_valor': p_valor,
        'interpretacion': interpretacion,
        'resultado': resultado}

In [ ]:
resultados = prueba_correlacion_spearman(df, 'ing_cor', 'gasto_mon')

## Ejercicio

Utilizando el mismo dataframe del ejemplo, realiza una prueba de hipótesis para la correlación para evaluar si existe una relación significativa entre el ingreso y alguna de las variables de gasto.

In [ ]:
df.info()

## Lecturas recomendadas

- El caso del "datasaurio" ilustra bien la importancia de graficar las variables: https://www.scientificamerican.com/article/what-this-graph-of-a-dinosaur-can-teach-us-about-doing-better-science/
- Para una discusión de cómo tomar decisiones a partir de correlaciones es recomendable la siguiente lectura: https://hbr.org/2014/03/when-to-act-on-a-correlation-and-when-not-to